# Analyse results of benchmark

And because i love it, we will do it using SQL & duckdb, yeah!

In [4]:
import duckdb
import pandas as pd
import plotly.express as px     # my favourite viz package 

In [ ]:
# we create a reusable table for our analysis 
duckdb.sql(f"""
CREATE OR REPLACE TABLE results AS
SELECT * FROM read_json('./data/benchmark_results.json');
""")

In [21]:
# Summarize the results (equivalent of .describe method in python)
duckdb.sql('summarize results;')

┌──────────────┬─────────────┬─────────────┬─────────────────────────┬───────────────┬────────────────────┬───────────────────┬─────────────┬────────────────────┬─────────────┬───────┬─────────────────┐
│ column_name  │ column_type │     min     │           max           │ approx_unique │        avg         │        std        │     q25     │        q50         │     q75     │ count │ null_percentage │
│   varchar    │   varchar   │   varchar   │         varchar         │     int64     │      varchar       │      varchar      │   varchar   │      varchar       │   varchar   │ int64 │  decimal(9,2)   │
├──────────────┼─────────────┼─────────────┼─────────────────────────┼───────────────┼────────────────────┼───────────────────┼─────────────┼────────────────────┼─────────────┼───────┼─────────────────┤
│ num_players  │ BIGINT      │ 23          │ 100000                  │             5 │ 18528.166666666668 │ 37052.62710604134 │ 46          │ 550                │ 10000       │    42 │    

In [18]:
# Quick look on average time and memory usage
duckdb.sql(f"""
SELECT 
    function,
    round(avg(time_in_s),4) as avg_time_in_s,
    round(avg(memory_in_mb), 4) as avg_memory_in_mb,
FROM results
GROUP BY ALL
ORDER BY avg_time_in_s
""")

┌─────────────────────────┬───────────────┬──────────────────┐
│        function         │ avg_time_in_s │ avg_memory_in_mb │
│         varchar         │    double     │      double      │
├─────────────────────────┼───────────────┼──────────────────┤
│ unpack_operator_flatten │        0.0396 │           15.565 │
│ generator_flatten       │        0.3773 │          15.5778 │
│ manual_flatten          │        0.4006 │          15.5658 │
│ polars_flatten          │        0.5242 │          23.8617 │
│ pandas_flatten          │        2.4315 │          67.9432 │
│ dlt_flatten             │        2.8233 │          15.1456 │
│ flatdict_flatten        │        4.6799 │           35.135 │
└─────────────────────────┴───────────────┴──────────────────┘

In [26]:
# check the memory usage
duckdb.sql("""
SELECT 
    function,
    round(min(memory_in_mb),4) as min_memory_in_mb,
    round(max(memory_in_mb),4) as max_memory_in_mb,
    round(avg(memory_in_mb), 4) as avg_memory_in_mb,
    round(median(memory_in_mb), 4) as median_memory_in_mb
FROM results
GROUP BY ALL
ORDER BY median_memory_in_mb 
""")

┌─────────────────────────┬──────────────────┬──────────────────┬──────────────────┬─────────────────────┐
│        function         │ min_memory_in_mb │ max_memory_in_mb │ avg_memory_in_mb │ median_memory_in_mb │
│         varchar         │      double      │      double      │      double      │       double        │
├─────────────────────────┼──────────────────┼──────────────────┼──────────────────┼─────────────────────┤
│ unpack_operator_flatten │           0.0195 │          84.0011 │           15.565 │              0.4626 │
│ manual_flatten          │           0.0203 │          84.0019 │          15.5658 │              0.4634 │
│ generator_flatten       │           0.0457 │          84.0122 │          15.5778 │              0.4735 │
│ polars_flatten          │           0.0411 │          128.785 │          23.8617 │              0.7061 │
│ flatdict_flatten        │           0.0665 │         189.5238 │           35.135 │              1.0656 │
│ pandas_flatten          │          

In [ ]:
# check the time spent
duckdb.sql("""
SELECT 
    function,
    round(min(time_in_s),4) as min_time_in_s,
    round(max(time_in_s),4) as max_time_in_s,
    round(avg(time_in_s), 4) as avg_time_in_s,
    round(median(time_in_s), 4) as median_time_in_s
FROM results
GROUP BY ALL
ORDER BY median_time_in_s
""")

┌─────────────────────────┬───────────────┬───────────────┬───────────────┬──────────────────┐
│        function         │ min_time_in_s │ max_time_in_s │ avg_time_in_s │ median_time_in_s │
│         varchar         │    double     │    double     │    double     │      double      │
├─────────────────────────┼───────────────┼───────────────┼───────────────┼──────────────────┤
│ unpack_operator_flatten │           0.0 │        0.2221 │        0.0396 │           0.0008 │
│ generator_flatten       │        0.0006 │        2.0507 │        0.3773 │           0.0103 │
│ manual_flatten          │        0.0006 │        2.1678 │        0.4006 │           0.0115 │
│ polars_flatten          │        0.0015 │        2.8321 │        0.5242 │           0.0246 │
│ pandas_flatten          │        0.0078 │       13.1588 │        2.4315 │           0.0651 │
│ flatdict_flatten        │        0.0061 │       25.3037 │        4.6799 │           0.1413 │
│ dlt_flatten             │        0.0282 │       

In [28]:
duckdb.sql("""
SELECT 
    num_players,
    function,
    min(time_in_s) OVER(PARTITION BY num_players ORDER BY time_in_s ) as min_time,
    max(time_in_s) OVER(PARTITION BY num_players ORDER BY time_in_s ) as max_time,
    avg(time_in_s) OVER(PARTITION BY num_players ORDER BY time_in_s ) as avg_time_in_s,
FROM results
ORDER BY avg_time_in_s DESC;
""")

┌─────────────┬─────────────────────────┬─────────────┬───────────────┬────────────────────────┐
│ num_players │        function         │  min_time   │   max_time    │     avg_time_in_s      │
│    int64    │         varchar         │   double    │    double     │         double         │
├─────────────┼─────────────────────────┼─────────────┼───────────────┼────────────────────────┤
│      100000 │ flatdict_flatten        │ 0.222108792 │ 25.3036541659 │      8.666921458285714 │
│      100000 │ dlt_flatten             │ 0.222108792 │  14.933201167 │      5.894132673683333 │
│      100000 │ pandas_flatten          │ 0.222108792 │  13.158782875 │          4.08631897502 │
│      100000 │ polars_flatten          │ 0.222108792 │  2.8321386661 │         1.818203000025 │
│      100000 │ manual_flatten          │ 0.222108792 │      2.167825 │     1.4802244446666666 │
│      100000 │ generator_flatten       │ 0.222108792 │   2.050739542 │            1.136424167 │
│       10000 │ flatdict_flatt

# Visualizations

In [30]:
# we create a dataframe from our duckdb object to create vizs
all_data = duckdb.sql("FROM results;")
df = all_data.df()
all_data.columns

['num_players', 'function', 'time_in_s', 'memory_in_mb']

In [34]:
# define the template variables to be reused 
template='plotly_dark'
height=800
width=1200
labels={
        'num_players': 'Number of players',
        'memory_in_mb': 'Memory usage (MB)',
        'function': 'Functions',
        'time_in_s': 'Execution time (seconds)'
    }

## Memory usage & execution time relation

In [ ]:
# scatter plot memory usage based on executioen time for the most data point( 100k players)

df_max_players = df.loc[df['num_players'] >= 100000].copy()     # create a copy with only results from 

df_max_players['time_in_s'] = df_max_players['time_in_s'].round(2)
df_max_players['memory_in_mb'] = df_max_players['memory_in_mb'].round(2) 

h_bar = px.scatter(
    df_max_players,
    title='Relation memory usage & execution time for 100k players (log scale)',
    x='time_in_s',
    y='memory_in_mb',
    color='function',
    size='memory_in_mb',
    size_max=50,
    log_x=True,
    log_y=True,
    template=template,
    labels=labels,
    height=height,
    #width=width
)

h_bar.show()


## Memory usage


In [38]:
import plotly.express as px
import pandas as pd

# Assuming all_data is your DataFrame with columns: function, num_players, memory_in_mb
df = all_data.df()

# Create a line chart
fig = px.line(
    df,
    x='num_players',
    y='memory_in_mb',
    color='function',
    markers=True,  
    title='Memory Usage by Number of Players (log scale)',
    labels=labels,
    log_x=True,
    log_y=True,
    template=template,
    height=height,
    width=width
)

fig.show()


In [39]:
bar_time = px.bar(
    df_max_players.sort_values('memory_in_mb'),
    title='Memory usage for 100k players (in MB)',
    labels=labels,
    x='memory_in_mb',
    y='function',
    color='function',
    #barmode='group',
    text_auto='.3s',
    orientation='h',
    template=template,
    height=height,
    width=width
)

bar_time.show()

In [40]:
# create a df with only 100 & 100k players columns
df_hundred = df.loc[df['num_players'].isin([100, 100_000])]

# Create a grouped bar chart
fig = px.bar(
    df_hundred.sort_values('memory_in_mb'),
    x='function',
    y='memory_in_mb',
    color='function',
    text_auto='.2s',
    facet_col='num_players',  # Create separate subplot for each player count
    facet_col_wrap=2,  # Number of subplots per row
    title='Memory Usage by Function for Different Player Counts',
    labels=labels,
    template=template,
    height=height,
    width=width,
   
)
# Update facet titles to show player count more clearly
fig.for_each_annotation(lambda a: a.update(text=f"{a.text.split('=')[1]} players"))

# Adjust y-axis ranges independently
for i, num_players in enumerate([100, 100_000]):
    fig.update_yaxes(matches=None)  # Disable y-axis matching

fig.show()


In [45]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Filter data for the two player counts
df_100 = df[df['num_players'] == 100].copy()

# Sort both dataframes by the same order (e.g., by function name)
df_100 = df_100.sort_values('function')
df_max_players = df_max_players.sort_values('function')

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])


# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces for 100 players (left y-axis)
fig.add_trace(
    go.Scatter(
        x=df_100['function'],
        y=df_100['memory_in_mb'],
        name="100 Players",
        mode='lines+markers',
        marker=dict(size=15),
        line=dict(width=2, color='pink')
    ),
    secondary_y=False,
)

# Add traces for 100,000 players (right y-axis)
fig.add_trace(
    go.Scatter(
        x=df_max_players['function'],
        y=df_max_players['memory_in_mb'],
        name="100,000 Players",
        mode='lines+markers',
        marker=dict(size=15),
        line=dict(width=2, color='red')
    ),
    secondary_y=True,
)

# Update layout
fig.update_layout(
    title_text="Memory Usage Comparison: 100 vs 100,000 Players",
    template=template,
    height=height,
    width=width
)

# Set y-axes titles
fig.update_yaxes(title_text="Memory (MB) - 100 Players", secondary_y=False)
fig.update_yaxes(title_text="Memory (MB) - 100,000 Players", secondary_y=True)

fig.show()


## Execution Time 


In [42]:
bar_time = px.bar(
    df_hundred.sort_values(by='time_in_s'),
    x='time_in_s',
    y='function',
    color='function',
    #barmode='group',
    text_auto='.3s',
    orientation='h',
    template=template,
    height=height,
    width=width
)

bar_time.show()

In [51]:
# Calculate time per player
df_100['time_per_player'] = df_100['time_in_s'] / 100
df_max_players['time_per_player'] = df_max_players['time_in_s'] / 100_000

# Create figure
fig = go.Figure()

# Add traces for time per player
fig.add_trace(
    go.Bar(
        x=df_100['function'],
        y=df_100['time_per_player'],
        name="100 Players",
        marker_color='pink',
        opacity=0.7
    )
)

fig.add_trace(
    go.Bar(
        x=df_max_players['function'],
        y=df_max_players['time_per_player'],
        name="100,000 Players",
        marker_color='red',
        opacity=0.7
    )
)

# Update layout
fig.update_layout(
    title_text="Time Efficiency (Seconds per Player)",
    template='plotly_dark',
    height=height,
    width=width,
    barmode='group',
    yaxis=dict(title="Time per Player (s)"),
    xaxis=dict(title="Function"),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig.show()


---

# Bonus: Analyse the loop style

In [97]:
duckdb.sql("""
SELECT 
    function,
    round(avg(time_in_s),4) as avg_time_in_s,
    round(median(time_in_s),4) as median_time_in_s,
    round(avg(memory_in_mb), 4) as avg_memory_in_mb,
    round(median(memory_in_mb), 4) as median_memory

FROM read_json('data/compare_loops_results.json')
GROUP BY 1
ORDER BY avg_time_in_s 
""")

┌─────────────────────────────────────┬───────────────┬──────────────────┬──────────────────┬───────────────┐
│              function               │ avg_time_in_s │ median_time_in_s │ avg_memory_in_mb │ median_memory │
│               varchar               │    double     │      double      │      double      │    double     │
├─────────────────────────────────────┼───────────────┼──────────────────┼──────────────────┼───────────────┤
│ flatdict_flatten_gen_comprehension  │         0.121 │           0.0057 │          18.6758 │         0.847 │
│ flatdict_flatten_list_comprehension │        0.1298 │           0.0056 │          18.6754 │        0.8465 │
│ flatdict_flatten                    │        5.1377 │           0.2339 │          42.1424 │        1.9194 │
└─────────────────────────────────────┴───────────────┴──────────────────┴──────────────────┴───────────────┘